In [22]:
from numpy import info
import requests
from requests import Request, Session
import numpy as np
import io
import pandas as pd
from datetime import datetime
import json
from aioconnect.helpers import *
from dotenv import load_dotenv
from os import getenv

In [3]:
import aioconnect

In [4]:
def get_token(
    email: str,
    password: str,
):
    """Log into AIO Impact and get a token.

    Parameters
    ----------
    email : str
        Email address of the user account.

    password : str
        Password of the user account.

    Returns
    -------
    str
        Bearer token.

    Raises
    -------
    requests.exceptions.HTTPError
        If the username and password are wrong.

    Examples
    --------
    >>> aioconnect.get_token(
    >>>     email="firstname.lastname@aioneers.com", password="xxx",
    >>> )
    """

    url = getenv("CONNECT_URL") + "/login"
    payload = json.dumps({"email": email, "password": password})
    headers = {"Content-Type": "application/json"}

    response = requests.request("POST", url, headers=headers, data=payload)
    response.raise_for_status()

    token = response.json()["token"]

    return token

In [5]:
def test_correct_credentials():
    password = aioconnect.vault_get_secret(
        scope="aio-data-science-key", key="sebastian-szilvas-aio-impact"
    )

    res = get_token(
        email="sebastian.szilvas@aioneers.com",
        password=f"{password}",
    )

    assert isinstance(res, str)
    assert len(res) > 250

def test_wrong_password():
    try:
        res = get_token(
            email="sebastian.szilvas@aioneers.com",
            password="wrong password",
        )
    except requests.exceptions.HTTPError as exception:
        assert exception.response.status_code == 401


In [5]:
test_correct_credentials()
test_wrong_password()

In [24]:
def convert_to_json(df):
    """
    The function will convert the pandas dataframe to json file.
    -----------------------------

    Parameters : 
    -----------------------------
    df -> Pandas.DataFrame : It is the dataframe which you want to convert into json format.

    Return : 
    -----------------------------
    output -> json : It is the data converted into json format. 

    """
    # Checking Conditions
    if df['externalId'].isna().any() or df['name'].isna().any() or df['actuals value'].isna().any() or df['actuals timestamp'].isna().any():
        raise ValueError('Column Names : externalId, name, actuals value and actuals timestamp cannot be kept empty.')
    
    df_2 = df.drop(['externalId', 'name', 'description', 'actuals value', 'actuals timestamp', 'target value',
        'target timestamp', 'projections value', 'projections timestamp'], axis=1)

    m_notnull = pd.notnull(df_2["measureId"])

    df_3 = df_2[m_notnull]

    if df_3['initiativeId'].isna().any():
        raise ValueError('If measureId is provided then initiativeId is mandatory.')
    
    i_null = pd.isnull(df_2["initiativeId"])
    
    df_4 = df_2[i_null]

    if df_4['metricId'].isna().any():
        raise ValueError('If initiativeId is not provided then metricId should be provided.')
    
    # Grouping columns
    finalList = []
    grouped = df.groupby(['externalId'])

    for key, value in grouped:
        
        dictionary = {}

        j = grouped.get_group(key).reset_index(drop=True)
        dictionary['externalId'] = j.at[0, 'externalId']
        dictionary['name'] = j.at[0, 'name']
        dictionary['description'] = j.at[0, 'description']
        dictionary['initiativeId'] = j.at[0, 'initiativeId']
        dictionary['measureId'] = j.at[0, 'measureId']
        dictionary['metricId'] = j.at[0, 'metricId']
        
        dict_actual = []
        dict_target = []
        dict_projection = []
        
        anotherDict_actual = {}
        anotherDict_target = {}
        anotherDict_projections = {}
        
        for i in j.index:

            anotherDict_actual['value'] = j.at[i, 'actuals value']
            anotherDict_actual['timestamp'] = j.at[i, 'actuals timestamp']
            
            anotherDict_target['value'] = j.at[i, 'target value']
            anotherDict_target['timestamp'] = j.at[i, 'target timestamp']
            
            anotherDict_projections['value'] = j.at[i, 'projections value']
            anotherDict_projections['timestamp'] = j.at[i, 'projections timestamp']
            
            dictionary_copy_actual = anotherDict_actual.copy()
            dict_actual.append(dictionary_copy_actual)

            dictionary_copy_target = anotherDict_target.copy()
            dict_target.append(dictionary_copy_target)
            
            dictionary_copy_projection = anotherDict_projections.copy()
            dict_projection.append(dictionary_copy_projection)
            
        dictionary['actuals'] = dict_actual
        dictionary['target'] = dict_target
        dictionary['projections'] = dict_projection
        
        dict_copy = dictionary.copy()
        finalList.append(dict_copy)

    # Dump the data into output variable
    output = json.dumps(finalList, sort_keys = True, indent=4, cls=NumpyEncoder)
    
    return output

class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):

            return int(obj)

        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)

        elif isinstance(obj, (np.complex_, np.complex64, np.complex128)):
            return {'real': obj.real, 'imag': obj.imag}

        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()

        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None

        return json.JSONEncoder.default(self, obj)

In [25]:
def upsert_DOT(
    token: str,
    dataframe: pd.DataFrame,
    limit: int
) -> list:
    """Create a new DOT in AIO Impact or update it if the DOT is already existing.

    Parameters
    ----------
    token : str
        Token which was returned from the user login.

    dataframe : Pandas.DataFrame
        Dataframe containing DOT details.
    
    limit : int
        integer value that indicates minimum value of DOTs to be sent.

    Returns
    -------
    requests.Response
        HTTP response.

    Examples
    --------
    >>> token = aioconnect.get_token(
    >>> email="firstname.lastname@aioneers.com", password="xxx",
    >>> )
    >>> res = aioconnect.upsert_DOT(
    >>>     token=token,
    >>>     dataframe = df
    >>> )
    """

    url = getenv("CONNECT_URL") + "/dots"

    # columns_list = [
    #     "externalId",
    #     "name",
    #     "description",
    #     "metricId",
    #     "actuals",
    # ]

    # if all([item in dataframe.columns for item in columns_list]):
    for i in range(0, len(dataframe), limit):
        slc = dataframe.iloc[i : i + limit]
        payload = convert_to_json(slc)


        response = requests.put(
        url=url, headers={"Authorization": f"Bearer {token}",
                            'Content-Type': 'application/json'}, data=payload)
        response.raise_for_status()

        return response
    # else:
    #     raise KeyError("Columns not correct")

In [29]:
password = aioconnect.vault_get_secret(
        scope="aio-data-science-key", key="sebastian-szilvas-aio-impact"
    )

token = get_token(
    email="sebastian.szilvas@aioneers.com",
    password=f"{password}",
)


def test_single_DOT_correct_headers():
    data = [{
        "externalId": "TESTANALYSEQC",
        "name": "Test1",
        "description": "This is a test dot.",
        "initiativeId": "cd0eef61-5d45-4753-8436-ed12e113a94a",
        "measureId": "6a8ab3a1-6a32-494b-a147-5c900c750d98",
        "metricId": 101,
        "actuals value": 66,
        "actuals timestamp": "2021-07-30T00:00:00+0000",
        "target value": 50,
        "target timestamp": "2021-07-30T00:00:00+0000",
        "projections value": 55,
        "projections timestamp": "2021-07-30T00:00:00+0000",
        },
        {
        "externalId": "TESTANALYSEQC2",
        "name": "Test2",
        "description": "This is a test dot.",
        "initiativeId": "cd0eef61-5d45-4753-8436-ed12e113a94a",
        "measureId": "6a8ab3a1-6a32-494b-a147-5c900c750d98",
        "metricId": 101,
        "actuals value": 69,
        "actuals timestamp": "2021-07-30T00:00:00+0000",
        "target value": 51,
        "target timestamp": "2021-07-30T00:00:00+0000",
        "projections value": 56,
        "projections timestamp": "2021-07-30T00:00:00+0000"
        },
        {
        "externalId": "TESTANALYSEQC3",
        "name": "Test3",
        "description": "This is a test dot.",
        "initiativeId": "cd0eef61-5d45-4753-8436-ed12e113a94a",
        "measureId": "6a8ab3a1-6a32-494b-a147-5c900c750d98",
        "metricId": 101,
        "actuals value": 70,
        "actuals timestamp": "2021-07-30T00:00:00+0000",
        "target value": 52,
        "target timestamp": "2021-07-30T00:00:00+0000",
        "projections value": 57,
        "projections timestamp": "2021-07-30T00:00:00+0000"
        },
        {
        "externalId": "TESTANALYSEQC4",
        "name": "Test5",
        "description": "This is a test dot.",
        "initiativeId": "cd0eef61-5d45-4753-8436-ed12e113a94a",
        "measureId": "6a8ab3a1-6a32-494b-a147-5c900c750d98",
        "metricId": 101,
        "actuals value": 71,
        "actuals timestamp": "2021-07-30T00:00:00+0000",
        "target value": 53,
        "target timestamp": "2021-07-30T00:00:00+0000",
        "projections value": 58,
        "projections timestamp": "2021-07-30T00:00:00+0000"
        },
        {
        "externalId": "TESTANALYSEQC5",
        "name": "Test5",
        "description": "This is a test dot.",
        "initiativeId": "cd0eef61-5d45-4753-8436-ed12e113a94a",
        "measureId": "6a8ab3a1-6a32-494b-a147-5c900c750d98",
        "metricId": 101,
        "actuals value": 73,
        "actuals timestamp": "2021-07-30T00:00:00+0000",
        "target value": 55,
        "target timestamp": "2021-07-30T00:00:00+0000",
        "projections value": 59,
        "projections timestamp": "2021-07-30T00:00:00+0000"
        }
    ]

    # column_names = ["externalId", "name","description","initiativeId","measureId","metricId","actuals value",
    # "actuals timestamp","target value","target timestamp","projections value","projections timestamp"]
    
    df = pd.DataFrame.from_dict(data)

    # print(df.info())


    res = upsert_DOT(
        token=token, dataframe=df, limit = 2)

    assert res.status_code == 200
    assert (res.json()["created"] == len(df.index) or
            res.json()["updated"] == len(df.index))


def test_two_DOTs_correct_headers():
    data = {
                "externalId": ["2343284","73894792"],
                "name": ["helloooooo","helloooooo2"],
                "description": ["sdklfjslfdjsldf","sdklfjslfdjsldf"],
                "metricId": [10,10],
                "actuals": [[
                    {
                            "timestamp": "2021-12-07T14:59:38.273Z",
                            "value": 15
                    }
                ],
                [
                    {
                            "timestamp": "2021-12-07T14:59:38.273Z",
                            "value": 17
                    }
                ]],
            }
    df = pd.DataFrame.from_dict(data)

    res = upsert_DOT(token = token,
        dataframe= df,
    )

    assert res.status_code == 200
    assert (res.json()["created"] == len(df.index) or
            res.json()["updated"] == len(df.index))

In [30]:
test_single_DOT_correct_headers()

HTTPError: 502 Server Error: BAD GATEWAY for url: https://dev2-connect.aioneers.tech/dots

In [24]:
test_single_DOT_correct_headers()
test_two_DOTs_correct_headers()

  externalId         name      description  metricId  \
0    2343284   helloooooo  sdklfjslfdjsldf        10   
1   73894792  helloooooo2  sdklfjslfdjsldf        10   

                                             actuals  
0  [{'timestamp': '2021-12-07T14:59:38.273Z', 'va...  
1  [{'timestamp': '2021-12-07T14:59:38.273Z', 'va...  


# This function still needs fixing

In [ ]:
# def get_object(
#     token: str,
#     object: str = "digitalObjectTwins",
# ) -> list:
#     """Get JSON object.

#     Parameters
#     ----------
#     token : str
#         Token which was returned from the user login.
#     object : str = "dotTypes"
#         Object to be extracted from the API.

#     Returns
#     -------
#     list
#         List of JSON objects.

#     Raises
#     ------
#     KeyError
#         Raises KeyError when the input is not correct.
#     """
#     url = getenv("CONNECT_URL")

#     if object == "digitalObjectTwins":
#         url += "/digitalObjectTwins/"
#     elif object == "measures":
#         url += "/measures/"
#     elif object == "initiatives":
#         url += "/initiatives/"
#     else:
#         raise KeyError

#     headers = {"Authorization": f"Bearer {token}"}

#     response = requests.request("GET", url, headers=headers)
#     response.raise_for_status()
#     return response.json()